# Part 0 & 1

# Load dataset

In [1]:
from IPython import embed
from datasets import load_dataset
from keras.src.layers import average

dataset = load_dataset('rotten_tomatoes')
train_dataset = dataset['train']
validation_dataset = dataset['validation']
test_dataset = dataset['test']

C:\Users\Aozy\anaconda3\envs\sc4002\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_df = train_dataset.to_pandas()
train_df.head()

,text,label
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1


# Create vocab from training dataset

In [3]:
import nltk


def build_vocab(train_dataset):
    # Create set, unique words only
    vocab = set()
    train_dataset_pos = []
    
    # Loop thru each sentence in training dataset
    for sentence in train_dataset['text']:
        # Basic text processing
        
        # Case folding
        sentence = sentence.lower()
        
        # NLTK tokenizer does a good job at separating meaningful words + punctuations
        # Better than defining regex ourselves
        word_list = nltk.tokenize.word_tokenize(sentence)
        
        # Further split words into separate words
        # e.g., 'well-being' -> 'well', 'being'
        # e.g., 'music/song' -> 'music', 'song'
        split_word_list = []
        for word in word_list:
            # If the word contains a hyphen or a slash, split it further
            if '-' in word or '/' in word:
                split_word_list.extend(word.replace('-', ' ').replace('/', ' ').split())
            else:
                split_word_list.append(word)
        
        # Dont remove all special characters, some are meaningful
        # Some words are surrounded by single/double quotes
        final_word_list = [word.strip("'\"") for word in split_word_list]
        
        # Add into set
        vocab.update(final_word_list)
        
        # Get pos tags
        # Also build POS tags
        pos_tags = nltk.pos_tag(final_word_list)
        train_dataset_pos.append(pos_tags)
        
    vocab.discard('')
    return vocab, train_dataset_pos

In [4]:
vocab, train_dataset_pos = build_vocab(train_dataset)

print(vocab)

{'sociological', 'corruscating', 'century', 'kegger', 'slog', 'event', 'congeals', 'canvas', 'sacrifices', 'boasting', 'simpson', 'beijing', 'confessional', 'nickelodeon', 'romance', 'synthetic', 'unremittingly', 'balding', 'na', 'enlightenment', 'decides', 'susan', 'confidently', 'capability', 'cut', 'wertmuller', 'blows', 'mau', 'smaller', 'sophisticates', 'instrument', 'shoes', 'principal', 'yvan', 'killed', 'traps', 'staggering', 'defies', 'chop', 'sabrina', 'well', 'swedish', 'twinkly', 'pawn', 'brains', 'limbo', 'awakening', 'betrayal', 'applies', 'admirer', 'oddest', 'launching', 'anticipation', 'miserable', 'cribbing', 'howler', 'bruised', 'buffoons', 'prism', 'anthropologically', 'sticky', 'overcome', 'hanukkah', 'constantly', 'regular', 'recite', 'paints', 'contours', 'cerebral', 'fit', 'actual', 'tenderness', 'últimos', 'develops', 'trumpets', 'purest', 'nia', 'annoyances', 'pubescent', 'capably', 'contemplation', 'buying', 'untugged', 'dictums', 'gooeyness', 'hippies', 'te'

In [5]:
print(train_dataset_pos[0])

[('the', 'DT'), ('rock', 'NN'), ('is', 'VBZ'), ('destined', 'VBN'), ('to', 'TO'), ('be', 'VB'), ('the', 'DT'), ('21st', 'JJ'), ('century', 'NN'), ('s', 'VBD'), ('new', 'JJ'), ('``', '``'), ('conan', 'JJ'), ('``', '``'), ('and', 'CC'), ('that', 'IN'), ('he', 'PRP'), ('s', 'VBZ'), ('going', 'VBG'), ('to', 'TO'), ('make', 'VB'), ('a', 'DT'), ('splash', 'NN'), ('even', 'RB'), ('greater', 'JJR'), ('than', 'IN'), ('arnold', 'RB'), ('schwarzenegger', 'NN'), (',', ','), ('jean', 'JJ'), ('claud', 'NN'), ('van', 'NN'), ('damme', 'NN'), ('or', 'CC'), ('steven', 'JJ'), ('segal', 'NN'), ('.', '.')]


## (a) What is the size of the vocabulary formed from your training data?

In [6]:
print(f'Vocab size/Unique words: {len(vocab)}')

Vocab size/Unique words: 16535


# Part 1: Preparing Word Embeddings

## Download GloVe embeddings: https://nlp.stanford.edu/projects/glove/
- Uncased means all words are lowercase

In [7]:
# Load GloVe embeddings
import numpy as np

def load_glove_embeddings(path):
    glove_embeddings = {}
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float64')
            glove_embeddings[word] = vector
            
    return glove_embeddings

In [8]:
glove_embeddings = load_glove_embeddings('glove.6B.300d.txt')

In [9]:
print(f'Glove embedding matrix for "the":\n {glove_embeddings["the"]}')

Glove embedding matrix for "the":
 [ 4.6560e-02  2.1318e-01 -7.4364e-03 -4.5854e-01 -3.5639e-02  2.3643e-01
 -2.8836e-01  2.1521e-01 -1.3486e-01 -1.6413e+00 -2.6091e-01  3.2434e-02
  5.6621e-02 -4.3296e-02 -2.1672e-02  2.2476e-01 -7.5129e-02 -6.7018e-02
 -1.4247e-01  3.8825e-02 -1.8951e-01  2.9977e-01  3.9305e-01  1.7887e-01
 -1.7343e-01 -2.1178e-01  2.3617e-01 -6.3681e-02 -4.2318e-01 -1.1661e-01
  9.3754e-02  1.7296e-01 -3.3073e-01  4.9112e-01 -6.8995e-01 -9.2462e-02
  2.4742e-01 -1.7991e-01  9.7908e-02  8.3118e-02  1.5299e-01 -2.7276e-01
 -3.8934e-02  5.4453e-01  5.3737e-01  2.9105e-01 -7.3514e-03  4.7880e-02
 -4.0760e-01 -2.6759e-02  1.7919e-01  1.0977e-02 -1.0963e-01 -2.6395e-01
  7.3990e-02  2.6236e-01 -1.5080e-01  3.4623e-01  2.5758e-01  1.1971e-01
 -3.7135e-02 -7.1593e-02  4.3898e-01 -4.0764e-02  1.6425e-02 -4.4640e-01
  1.7197e-01  4.6246e-02  5.8639e-02  4.1499e-02  5.3948e-01  5.2495e-01
  1.1361e-01 -4.8315e-02 -3.6385e-01  1.8704e-01  9.2761e-02 -1.1129e-01
 -4.2085e-01  1.

In [10]:
# Print size of the matrix
print(f'Number of unique words in embedding matrix: {len(glove_embeddings)}')

print(f'Number of Dimension/Features of Glove embedding matrix: {glove_embeddings["the"].shape[0]}')

Number of unique words in embedding matrix: 400000
Number of Dimension/Features of Glove embedding matrix: 300


## Create embedding matrix

In [11]:
# Finalize vocab
vocab_word_to_index = {word: idx for idx, word in enumerate(vocab)}

In [12]:
vocab_word_to_index

{'sociological': 0,
 'corruscating': 1,
 'century': 2,
 'kegger': 3,
 'slog': 4,
 'event': 5,
 'congeals': 6,
 'canvas': 7,
 'sacrifices': 8,
 'boasting': 9,
 'simpson': 10,
 'beijing': 11,
 'confessional': 12,
 'nickelodeon': 13,
 'romance': 14,
 'synthetic': 15,
 'unremittingly': 16,
 'balding': 17,
 'na': 18,
 'enlightenment': 19,
 'decides': 20,
 'susan': 21,
 'confidently': 22,
 'capability': 23,
 'cut': 24,
 'wertmuller': 25,
 'blows': 26,
 'mau': 27,
 'smaller': 28,
 'sophisticates': 29,
 'instrument': 30,
 'shoes': 31,
 'principal': 32,
 'yvan': 33,
 'killed': 34,
 'traps': 35,
 'staggering': 36,
 'defies': 37,
 'chop': 38,
 'sabrina': 39,
 'well': 40,
 'swedish': 41,
 'twinkly': 42,
 'pawn': 43,
 'brains': 44,
 'limbo': 45,
 'awakening': 46,
 'betrayal': 47,
 'applies': 48,
 'admirer': 49,
 'oddest': 50,
 'launching': 51,
 'anticipation': 52,
 'miserable': 53,
 'cribbing': 54,
 'howler': 55,
 'bruised': 56,
 'buffoons': 57,
 'prism': 58,
 'anthropologically': 59,
 'sticky': 60

In [13]:
def create_embedding_matrix(word_to_index, glove_embeddings):
    # Initialize embedding matrix with zeros
    # 50d
    embedding_matrix = np.zeros((len(vocab), 300), dtype='float64')
    
    # Loop thru each word in vocab
    for word, idx in word_to_index.items():
        # Check if word exists in glove embeddings
        if word in glove_embeddings:
            # Copy glove embedding to embedding matrix
            embedding_matrix[idx] = glove_embeddings[word]
            # If OOV, assign None first
            
    return embedding_matrix

In [14]:
embedding_matrix = create_embedding_matrix(vocab_word_to_index, glove_embeddings)

In [15]:
embedding_matrix

array([[ 0.28409 , -0.26594 ,  0.13595 , ...,  0.61216 ,  0.13325 ,
         0.10368 ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.27446 , -0.76348 , -0.26065 , ..., -0.22371 ,  0.041046,
         0.079926],
       ...,
       [ 0.25289 , -0.068793,  0.082297, ...,  0.22697 ,  0.06862 ,
        -0.17956 ],
       [ 0.32509 ,  0.01838 , -0.071966, ..., -0.12037 , -0.25952 ,
        -0.46926 ],
       [-0.042669, -0.23899 ,  0.51505 , ...,  0.097885, -0.31774 ,
         0.58296 ]])

## (b) We use OOV (out-of-vocabulary) to refer to those words appeared in the training data but not in the Word2vec (or Glove) dictionary. How many OOV words exist in your training data?

In [16]:
def get_oov_words(embedding_matrix, vocab_word_to_index):
    oov_words = []
    
    for word, idx in vocab_word_to_index.items():
        # Check if zero matrix
        if np.sum(embedding_matrix[idx]) == 0:
            oov_words.append(word)

    return oov_words

In [17]:
oov_words = get_oov_words(embedding_matrix, vocab_word_to_index)

In [18]:
oov_words.sort()
oov_words[:200]

['28k',
 '5ths',
 '8217',
 'abandone',
 'aborbing',
 'absolutamente',
 'aburrido',
 'acabamos',
 'accomodates',
 'aceitou',
 'achival',
 'achronological',
 'acontecimentos',
 'actorish',
 'actory',
 'actuación',
 'actuada',
 'addessi',
 'adorability',
 'adventues',
 'affirmational',
 "ain't",
 'alientation',
 'allodi',
 'amoses',
 'amusedly',
 'andamento',
 'animé',
 'anteing',
 'apallingly',
 'apesar',
 'aproveitar',
 'aqueles',
 "aren't",
 'arriesgado',
 'artnering',
 'artsploitation',
 'artístico',
 'assistir',
 'atacar',
 'atacarse',
 'atreve',
 'auteil',
 'autocritique',
 'b+',
 'bazadona',
 'bergmanesque',
 'beseechingly',
 'bibbidy',
 'bierbichler',
 'birot',
 'bizzarre',
 'bjorkness',
 'blighter',
 'blutarsky',
 'bobbidi',
 'bondish',
 'bornin',
 'bottomlessly',
 'bruckheimeresque',
 'brûlée',
 "bull's",
 'burningly',
 'bustingly',
 'butterfingered',
 'cadness',
 "cam'ron",
 'camareras',
 'cannier',
 'captivatingly',
 'capturou',
 'carente',
 'cativante',
 'certamente',
 'chabr

In [19]:
print(f"Number of OOV words: {len(oov_words)}")

Number of OOV words: 604


In [20]:
train_dataset_pos

[[('the', 'DT'),
  ('rock', 'NN'),
  ('is', 'VBZ'),
  ('destined', 'VBN'),
  ('to', 'TO'),
  ('be', 'VB'),
  ('the', 'DT'),
  ('21st', 'JJ'),
  ('century', 'NN'),
  ('s', 'VBD'),
  ('new', 'JJ'),
  ('``', '``'),
  ('conan', 'JJ'),
  ('``', '``'),
  ('and', 'CC'),
  ('that', 'IN'),
  ('he', 'PRP'),
  ('s', 'VBZ'),
  ('going', 'VBG'),
  ('to', 'TO'),
  ('make', 'VB'),
  ('a', 'DT'),
  ('splash', 'NN'),
  ('even', 'RB'),
  ('greater', 'JJR'),
  ('than', 'IN'),
  ('arnold', 'RB'),
  ('schwarzenegger', 'NN'),
  (',', ','),
  ('jean', 'JJ'),
  ('claud', 'NN'),
  ('van', 'NN'),
  ('damme', 'NN'),
  ('or', 'CC'),
  ('steven', 'JJ'),
  ('segal', 'NN'),
  ('.', '.')],
 [('the', 'DT'),
  ('gorgeously', 'RB'),
  ('elaborate', 'JJ'),
  ('continuation', 'NN'),
  ('of', 'IN'),
  ('``', '``'),
  ('the', 'DT'),
  ('lord', 'NN'),
  ('of', 'IN'),
  ('the', 'DT'),
  ('rings', 'NNS'),
  ('``', '``'),
  ('trilogy', 'NN'),
  ('is', 'VBZ'),
  ('so', 'RB'),
  ('huge', 'JJ'),
  ('that', 'IN'),
  ('a', 'DT'),
  

In [21]:
# Find POS tags for OOV words
oov_words_pos = {}
for oov_word in oov_words:
    # Find all possible POS for OOV word
    pos_tags = []
    for sentence_pos in train_dataset_pos:
        for word, pos in sentence_pos:
            if word == oov_word:
                pos_tags.append(pos)
    # # Assign
    oov_words_pos[oov_word] = pos_tags

In [22]:
oov_words_pos

{'28k': ['CD'],
 '5ths': ['CD'],
 '8217': ['CD', 'CD'],
 'abandone': ['NN'],
 'aborbing': ['VBG'],
 'absolutamente': ['NN'],
 'aburrido': ['NN'],
 'acabamos': ['NN'],
 'accomodates': ['VBZ'],
 'aceitou': ['NN'],
 'achival': ['JJ'],
 'achronological': ['NN', 'JJ'],
 'acontecimentos': ['NNS'],
 'actorish': ['JJ'],
 'actory': ['JJ'],
 'actuación': ['FW'],
 'actuada': ['NN', 'JJ'],
 'addessi': ['NN'],
 'adorability': ['NN'],
 'adventues': ['NNS'],
 'affirmational': ['JJ'],
 "ain't": ['IN'],
 'alientation': ['NN'],
 'allodi': ['JJ'],
 'amoses': ['NNS'],
 'amusedly': ['RB'],
 'andamento': ['NN'],
 'animé': ['JJ', 'NN'],
 'anteing': ['VBG'],
 'apallingly': ['RB'],
 'apesar': ['NN'],
 'aproveitar': ['NN'],
 'aqueles': ['NNS'],
 "aren't": ['JJ'],
 'arriesgado': ['FW'],
 'artnering': ['VBG'],
 'artsploitation': ['NN'],
 'artístico': ['JJ'],
 'assistir': ['IN'],
 'atacar': ['NN'],
 'atacarse': ['JJ'],
 'atreve': ['VBP'],
 'auteil': ['NN', 'JJ'],
 'autocritique': ['NN'],
 'b+': ['NN'],
 'bazadona'

## (c) The existence of the OOV words is one of the well-known limitations of Word2vec (or Glove). Without using any transformer-based language models (e.g., BERT, GPT, T5), what do you think is the best strategy to mitigate such limitation? Implement your solution in your source code. Show the corresponding code snippet.

## Pass 1: Use Stemming to match OOV words of different forms to the same word in GloVe

- Use snowball stemmer, as it is more aggressive than porter stemmer. Handles -ly words better

In [23]:
# from nltk.stem import PorterStemmer
# ps = PorterStemmer()

# from nltk.stem import SnowballStemmer
# ss = SnowballStemmer('english')

from nltk.stem import LancasterStemmer
ls = LancasterStemmer()

# Stem vocab
stemmed_glove_vocab = {ls.stem(word): word for word in glove_embeddings.keys()}

# Stem vocab and OOV words, find same word
def find_substitute_word_stem(oov_word, stemmed_glove_vocab):
    stemmed_oov_word = ls.stem(oov_word)
    if stemmed_oov_word in stemmed_glove_vocab.keys():
        return stemmed_glove_vocab[stemmed_oov_word]
    
    return None

In [24]:
# Copy substitute word embedding to OOV word
for oov_word in oov_words:
    best_substitute_word = find_substitute_word_stem(oov_word, stemmed_glove_vocab)
    if best_substitute_word:
        print(f'OOV word: {oov_word}, substitute word: {best_substitute_word}')
        embedding_matrix[vocab_word_to_index[oov_word]] = glove_embeddings[best_substitute_word]

OOV word: abandone, substitute word: abandonar
OOV word: accomodates, substitute word: accomodative
OOV word: achival, substitute word: acher
OOV word: actorish, substitute word: actionism
OOV word: actory, substitute word: actionism
OOV word: adorability, substitute word: adag
OOV word: affirmational, substitute word: affirmance
OOV word: alientation, substitute word: aliis
OOV word: allodi, substitute word: allodial
OOV word: amusedly, substitute word: amusa
OOV word: anteing, substitute word: antiseizure
OOV word: apallingly, substitute word: apala
OOV word: aqueles, substitute word: aquel
OOV word: atacar, substitute word: atac
OOV word: beseechingly, substitute word: beseeches
OOV word: birot, substitute word: birote
OOV word: bjorkness, substitute word: bjorke
OOV word: blighter, substitute word: blighting
OOV word: bondish, substitute word: bondian
OOV word: bottomlessly, substitute word: bottomless
OOV word: burningly, substitute word: burnable
OOV word: bustingly, substitute w

In [25]:
# Print remaining OOV words
oov_words = get_oov_words(embedding_matrix, vocab_word_to_index)

print(f"Number of OOV words left: {len(oov_words)}")

Number of OOV words left: 381


## Pass 2: Use Wordnet synonyms bank based on sentence POS tagging

In [26]:
oov_words_pos

{'28k': ['CD'],
 '5ths': ['CD'],
 '8217': ['CD', 'CD'],
 'abandone': ['NN'],
 'aborbing': ['VBG'],
 'absolutamente': ['NN'],
 'aburrido': ['NN'],
 'acabamos': ['NN'],
 'accomodates': ['VBZ'],
 'aceitou': ['NN'],
 'achival': ['JJ'],
 'achronological': ['NN', 'JJ'],
 'acontecimentos': ['NNS'],
 'actorish': ['JJ'],
 'actory': ['JJ'],
 'actuación': ['FW'],
 'actuada': ['NN', 'JJ'],
 'addessi': ['NN'],
 'adorability': ['NN'],
 'adventues': ['NNS'],
 'affirmational': ['JJ'],
 "ain't": ['IN'],
 'alientation': ['NN'],
 'allodi': ['JJ'],
 'amoses': ['NNS'],
 'amusedly': ['RB'],
 'andamento': ['NN'],
 'animé': ['JJ', 'NN'],
 'anteing': ['VBG'],
 'apallingly': ['RB'],
 'apesar': ['NN'],
 'aproveitar': ['NN'],
 'aqueles': ['NNS'],
 "aren't": ['JJ'],
 'arriesgado': ['FW'],
 'artnering': ['VBG'],
 'artsploitation': ['NN'],
 'artístico': ['JJ'],
 'assistir': ['IN'],
 'atacar': ['NN'],
 'atacarse': ['JJ'],
 'atreve': ['VBP'],
 'auteil': ['NN', 'JJ'],
 'autocritique': ['NN'],
 'b+': ['NN'],
 'bazadona'

In [27]:
from nltk.corpus import wordnet as wn

pos_mapping_dict = {'NN':wn.NOUN,
              'JJ':wn.ADJ,
              'VB':wn.VERB,
              'RB':wn.ADV,
              # NLTK does not have wn.ADV_SAT
              }

# Convert oov_words_pos to wordnet POS
def map_pos_list(pos_list, mapping_dict):
    mapped_pos = []
    for pos in pos_list:
        # Find the first matching POS tag in mapping_dict by first letter comparison
        matched_pos = next((mapping_dict[key] for key in mapping_dict if key[0] == pos[0]), None)
        mapped_pos.append(matched_pos)
    return mapped_pos

# Creating the pos list
oov_words_pos_wordnet = {}
for key, pos_list in oov_words_pos.items():
    mapped_pos_list = map_pos_list(pos_list, pos_mapping_dict)
    
    # Remove None from list
    cleaned_list = [pos for pos in mapped_pos_list if pos is not None]
    
    # If all nones, remove whole entry
    if cleaned_list:
        oov_words_pos_wordnet[key] = cleaned_list
    

# Print the new dictionary
for key, value in oov_words_pos_wordnet.items():
    print(f"{key}: {value}")

abandone: ['n']
aborbing: ['v']
absolutamente: ['n']
aburrido: ['n']
acabamos: ['n']
accomodates: ['v']
aceitou: ['n']
achival: ['a']
achronological: ['n', 'a']
acontecimentos: ['n']
actorish: ['a']
actory: ['a']
actuada: ['n', 'a']
addessi: ['n']
adorability: ['n']
adventues: ['n']
affirmational: ['a']
alientation: ['n']
allodi: ['a']
amoses: ['n']
amusedly: ['r']
andamento: ['n']
animé: ['a', 'n']
anteing: ['v']
apallingly: ['r']
apesar: ['n']
aproveitar: ['n']
aqueles: ['n']
aren't: ['a']
artnering: ['v']
artsploitation: ['n']
artístico: ['a']
atacar: ['n']
atacarse: ['a']
atreve: ['v']
auteil: ['n', 'a']
autocritique: ['n']
b+: ['n']
bazadona: ['n']
bergmanesque: ['a']
beseechingly: ['r']
bibbidy: ['n']
bierbichler: ['n']
birot: ['n', 'n', 'n', 'n', 'n', 'n']
bizzarre: ['a']
bjorkness: ['n']
blighter: ['n']
blutarsky: ['n']
bobbidi: ['n']
bondish: ['a']
bornin: ['n']
bottomlessly: ['r']
bruckheimeresque: ['a']
brûlée: ['n']
bull's: ['n']
burningly: ['r']
bustingly: ['r']
butterfing

In [28]:
def find_substitute_wordnet_synonym(oov_word, oov_words_pos_wordnet):
    # Find synonyms for OOV words
    # Retrieve POS tags list
    try:
        pos_list = oov_words_pos_wordnet[oov_word]
    except KeyError:
        return None
    
    # Find number of unique pos, except None
    unique_pos = list(set(pos_list))
    # sort by count
    unique_pos.sort(key=lambda x: pos_list.count(x), reverse=True)
    
    # Loop thru each unique pos, try to find synonyms
    for possible_pos_tag in unique_pos:
        # Get synonyms
        for synset in wn.synsets(oov_word, pos=possible_pos_tag):
            for lemma in synset.lemmas():
                if lemma.name() in glove_embeddings:
                    print(f'OOV word: {oov_word}, synonym: {lemma.name()}')
                    return glove_embeddings[lemma.name()]

In [29]:
for oov_word in oov_words:
    synonym_glove_embedding = find_substitute_wordnet_synonym(oov_word, oov_words_pos_wordnet)
    if synonym_glove_embedding is not None:
        embedding_matrix[vocab_word_to_index[oov_word]] = synonym_glove_embedding

OOV word: corniest, synonym: corny
OOV word: greasiest, synonym: greasy
OOV word: juiceless, synonym: dry
OOV word: stuffiest, synonym: airless
OOV word: excrescence, synonym: bulge
OOV word: shmear, synonym: schmear
OOV word: fuddled, synonym: befuddle
OOV word: perfervid, synonym: ardent


In [30]:
# Print remaining OOV words
oov_words = get_oov_words(embedding_matrix, vocab_word_to_index)

print(f"Number of OOV words left: {len(oov_words)}")

Number of OOV words left: 373


## Pass 3: Use Edit Distance to solve misspelled OOV words

In [31]:
from Levenshtein import distance as lev

# Find most similar word for OOV word
def find_substitute_word_edit_dist(oov_word, glove_embeddings):
    # Set to infinity first
    min_dist = float('inf')
    closest_word = None
    
    # Loop thru all words in glove embeddings
    for word in glove_embeddings:
        # Calculate edit distance
        dist = lev(oov_word, word)
        
        # Update if new minimum distance found
        if dist < min_dist:
            min_dist = dist
            closest_word = word
            
    return closest_word, min_dist

In [32]:
min_dist_thresh = 1

for oov_word in oov_words:
    best_substitute_word, min_dist = find_substitute_word_edit_dist(oov_word, glove_embeddings)
    if min_dist <= min_dist_thresh:
        print(f'OOV word: {oov_word}, substitute word: {best_substitute_word}, Distance: {min_dist}')
        # Copy substitute word embedding to OOV word
        embedding_matrix[vocab_word_to_index[oov_word]] = glove_embeddings[best_substitute_word]

OOV word: corruscating, substitute word: coruscating, Distance: 1
OOV word: últimos, substitute word: último, Distance: 1
OOV word: paródia, substitute word: parodia, Distance: 1
OOV word: lhe, substitute word: the, Distance: 1
OOV word: diretor, substitute word: director, Distance: 1
OOV word: costumey, substitute word: costume, Distance: 1
OOV word: makmalbaf, substitute word: makhmalbaf, Distance: 1
OOV word: saído, substitute word: sado, Distance: 1
OOV word: scuzbag, substitute word: scumbag, Distance: 1
OOV word: emotiva, substitute word: emotive, Distance: 1
OOV word: aborbing, substitute word: absorbing, Distance: 1
OOV word: fun's, substitute word: funds, Distance: 1
OOV word: episódio, substitute word: episodio, Distance: 1
OOV word: guión, substitute word: guinn, Distance: 1
OOV word: what, substitute word: what, Distance: 1
OOV word: flck, substitute word: flock, Distance: 1
OOV word: animé, substitute word: anime, Distance: 1
OOV word: hubac, substitute word: huac, Distan

# Remaining OOV words

In [33]:
# Print remaining OOV words
oov_words = get_oov_words(embedding_matrix, vocab_word_to_index)

print(f"Number of OOV words left: {len(oov_words)}")

Number of OOV words left: 210


## Pass 4: Split words into subwords and average their subword embeddings

In [34]:
# Move split point from left to right of the OOV word
def find_subword(oov_word, glove_embeddings):
    # Usually words dont go beyond 3 splits, but we only try 2 splits, 3 splits might lose too much meaning
    # Different meaning: slappingly -> slap ping ly
    for i in range(1, len(oov_word)):
        left = oov_word[:i]
        right = oov_word[i:]
        
        if left in glove_embeddings and right in glove_embeddings.keys() and wn.synsets(left) and wn.synsets(right):
            return left, right
        
    return None

In [35]:
for oov_word in oov_words:
    subwords_tup = find_subword(oov_word, glove_embeddings)
    if subwords_tup is not None:
        print(f'OOV word: {oov_word}, subwords: {subwords_tup}')
        embeddings = [glove_embeddings[subword] for subword in subwords_tup]
        avg_embeddings = np.mean(embeddings, axis=0)
        embedding_matrix[vocab_word_to_index[oov_word]] = avg_embeddings

OOV word: untugged, subwords: ('un', 'tugged')
OOV word: unreligious, subwords: ('un', 'religious')
OOV word: consegue, subwords: ('con', 'segue')
OOV word: dateflick, subwords: ('date', 'flick')
OOV word: unplundered, subwords: ('un', 'plundered')
OOV word: overemphatic, subwords: ('over', 'emphatic')
OOV word: unentertaining, subwords: ('un', 'entertaining')
OOV word: autocritique, subwords: ('auto', 'critique')
OOV word: flakeball, subwords: ('flake', 'ball')
OOV word: unconned, subwords: ('un', 'conned')
OOV word: unencouraging, subwords: ('un', 'encouraging')
OOV word: komediant, subwords: ('ko', 'mediant')
OOV word: surehanded, subwords: ('sure', 'handed')
OOV word: wisegirls, subwords: ('wise', 'girls')
OOV word: interspliced, subwords: ('inter', 'spliced')
OOV word: overplotted, subwords: ('over', 'plotted')
OOV word: mibii, subwords: ('mib', 'ii')
OOV word: squaddie, subwords: ('squad', 'die')
OOV word: superada, subwords: ('super', 'ada')
OOV word: overmanipulative, subwords:

# Remaining OOV words

In [36]:
# Print remaining OOV words
oov_words = get_oov_words(embedding_matrix, vocab_word_to_index)

print(f"Number of OOV words left: {len(oov_words)}")

Number of OOV words left: 165


In [37]:
oov_words

['shlockmeister',
 'glizty',
 'zzzzzzzzz',
 'actuada',
 'preciosista',
 'perseguição',
 'estafeta',
 'efteriades',
 'interações',
 'fílmica',
 'aproveitar',
 'direção',
 'camareras',
 'suspeito',
 'koshashvili',
 'incompetência',
 'splittingly',
 'satirizado',
 'atreve',
 'watstein',
 'bibbidy',
 'higuchinsky',
 'sychowski',
 'heremakono',
 'copmovieland',
 'provocatuers',
 'choquart',
 'uberviolence',
 'utilizar',
 'revigorates',
 'sorprenderá',
 'enfrentará',
 'diferença',
 'kibbitzes',
 'continuação',
 'addessi',
 'hamfisted',
 'prewarned',
 'frustrado',
 'projeção',
 'enternecedora',
 'blutarsky',
 'bierbichler',
 'originalidad',
 'melodramáticos',
 'ninguém',
 'emocionante',
 'gerbosi',
 'responsável',
 'wewannour',
 'kalesniko',
 'evolução',
 'roteirista',
 'unrecommendable',
 'simbolizando',
 'acabamos',
 'péssima',
 'flatula',
 'prechewed',
 'capturou',
 'bazadona',
 'incoloro',
 'pretenciosas',
 'condensada',
 'profundamente',
 'feardotcom',
 'shagster',
 'kazmierski',
 'bruck

## Last pass: Replace OOV words with <UNK> token

In [38]:
# Get min max range of glove embeddings
all_embeddings = np.stack(list(glove_embeddings.values()))
min_val = np.min(all_embeddings)
max_val = np.max(all_embeddings)

print(f"min: {min_val}")
print(f"max: {max_val}")

mean_embedding = np.mean(all_embeddings)
std_embedding = np.std(all_embeddings)

print(f"Mean: {mean_embedding}, Standard Deviation: {std_embedding}")

min: -3.0639
max: 3.2582
Mean: -0.003905011892532385, Standard Deviation: 0.3817702235076489


In [39]:
unk_token = '<UNK>'
# Random embedding for <UNK> token
embedding_dim = 300
unk_embedding = np.random.uniform(-0.25, 0.25, embedding_dim)

# Assign <UNK> token embedding to OOV words
oov_words = get_oov_words(embedding_matrix, vocab_word_to_index)
for oov_word in oov_words:
    ## Use a single random embedding for all OOV words
    embedding_matrix[vocab_word_to_index[oov_word]] = unk_embedding
    ## Zero embedding
    # embedding_matrix[vocab_word_to_index[oov_word]] = np.zeros(embedding_dim)
    
# Add <UNK> token to vocab and embedding matrix
vocab_word_to_index[unk_token] = len(vocab)
embedding_matrix = np.vstack([embedding_matrix, unk_embedding])

In [40]:
# embedding_matrix[17841]

## Alternative methods to test
- semantic similarity with cosine similarity
- use model to predict oov embeddings Mimick https://github.com/yuvalpinter/Mimick


# Finally, save embedding matrix and vocab_to_index mapping

In [41]:
import pickle


with open('embedding_matrix_300d.pkl', 'wb') as f:
    pickle.dump(embedding_matrix, f)
    
with open('vocab_word_to_index_300d.pkl', 'wb') as f:
    pickle.dump(vocab_word_to_index, f)